<a href="https://colab.research.google.com/github/zaidalyafeai/Notebooks/blob/master/TF4ST_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Foundation
import TensorFlow
import Python

## Download Data and Labels

In [2]:
let urllib = Python.import("urllib.request")
let fileBaseURL = "https://raw.githubusercontent.com/tensorflow/swift-models/stable/MNIST/"
let files = ["train-images-idx3-ubyte", "train-labels-idx1-ubyte"]

for file in files {
  print("downloading ... ", file)
  urllib.urlretrieve(fileBaseURL + file, filename: file)
}

downloading ...  train-images-idx3-ubyte
downloading ...  train-labels-idx1-ubyte


## Process Data 

In [0]:
var batchSize:Int32 = 32 

/// Reads a file into an array of bytes.
func readFile(_ path: String) -> [UInt8] {
    let url = URL(fileURLWithPath: path)
    let data = try! Data(contentsOf: url, options: [])
    return [UInt8](data)
}

/// Reads MNIST images and labels from specified file paths.
func readMNIST(imagesFile: String, labelsFile: String) -> (images: Tensor<Float>,
                                                           labels: Tensor<Int32>) {
    print("Reading data.")
    let images = readFile(imagesFile).dropFirst(16).map(Float.init)
    let labels = readFile(labelsFile).dropFirst(8).map(Int32.init)
    let rowCount = Int32(labels.count)
    let imageHeight: Int32 = 28, imageWidth: Int32 = 28

    print("Constructing data tensors.")
    return (
        images: Tensor(shape: [rowCount, 1, imageHeight, imageWidth], scalars: images)
            .transposed(withPermutations: [0, 2, 3, 1]) / 255, // NHWC
        labels: Tensor(labels)
    )
}

/// Split data into training and test
func splitTrainTest<Scalar>(data: Tensor<Scalar>, labels: Tensor<Scalar>) -> (Tensor<Scalar>, Tensor<Scalar>, Tensor<Scalar> , Tensor<Scalar>) {
  
  let N = Int32(data.shape[0])
  let split = Int32(0.8 * Float(N))
  
  let trainX = data[0..<split]
  let trainY = labels[0..<split]
  
  let testX = data[split..<N]
  let testY = labels[split..<N]
  
  return (trainX, trainY, testX, testY)
}

/// Extract a batch of certain size 
func minibatch<Scalar>(in x: Tensor<Scalar>, at index: Int32) -> Tensor<Scalar> {
    let start = Int32(index * batchSize)
    return x[start..<start+Int32(batchSize)]
}

In [4]:
// convert into tensors
let (data, trainNumericLabels) = readMNIST(imagesFile: files[0], labelsFile: files[1])
let labels = Tensor<Float>(oneHotAtIndices: trainNumericLabels, depth: 10)

// split into training and testing 
let (trainX, trainY, testX, testY) = splitTrainTest(data: data, labels: labels)

Reading data.
Constructing data tensors.


## CNN Model

In [0]:
struct CNN: Layer {
    var conv1 = Conv2D<Float>(filterShape: (3, 3, 1, 16), activation: relu) 
    var conv2 = Conv2D<Float>(filterShape: (3, 3, 16, 32), activation: relu) 
 
    var pool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
  
    var flatten = Flatten<Float>()
  
    var dense1 = Dense<Float>(inputSize: 5*5*32 , outputSize: 128, activation: tanh)
    var dense2 = Dense<Float>(inputSize: 128 , outputSize: 10)

    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
      var x = input
      x = conv1.applied(to: x, in: context)
      x = pool.applied(to: x, in: context)
      
      x = conv2.applied(to: x, in: context)
      x = pool.applied(to: x, in: context)
      
      x = flatten.applied(to: x, in: context)
      
      x = dense1.applied(to: x, in: context)
      x = dense2.applied(to: x, in: context)
      return x 
    }
}

let optimizer = SGD<CNN, Float>(learningRate: 0.01)

In [0]:
func getAccuracy(y:Tensor<Float>, logits:Tensor<Float>) -> Float{
  let yhat  = logits.argmax(squeezingAxis: 1) - y.argmax(squeezingAxis: 1)
  return Float(yhat.makeNumpyArray().count(where: { $0 == 0})) / Float(yhat.shape[0])
}

## Training

In [7]:
var model = CNN()

let trainingContext = Context(learningPhase: .training)
let inferenceContext = Context(learningPhase: .inference)

let stepsInEpoch:Int32 = Int32(Float(testX.shape[0]) / Float(batchSize))
var avgLoss:Float = 0.0
var avgAcc :Float = 0.0

for epoch in 0...4{
  
  //evaluate metrics
  avgLoss = 0.0
  avgAcc  = 0.0
    
  for i in 0..<stepsInEpoch {
  
    //get batches
    let X = minibatch(in: trainX, at: i)
    let y = minibatch(in: trainY, at: i)
    
    //calculate the loss and gradient
    let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
        let logits = model.applied(to: X, in: trainingContext)
        return softmaxCrossEntropy(logits: logits, oneHotLabels: y)
    }

    //make an optimizer step 
    optimizer.update(&model.allDifferentiableVariables, along: grads)    
    
    let logits = model.applied(to: X, in: inferenceContext)
    let acc = getAccuracy(y:y, logits:logits)
    
    avgLoss += (Float(loss) ?? 0.0)/Float(stepsInEpoch)
    avgAcc  += acc/Float(stepsInEpoch)
  }
  
  print(String(format:"epoch: %d, train_loss: %.2f, train_acc: %.2f", (epoch+1), avgLoss, avgAcc))
}

epoch: 1, train_loss: 1.48, train_acc: 0.62
epoch: 2, train_loss: 0.42, train_acc: 0.92
epoch: 3, train_loss: 0.30, train_acc: 0.95
epoch: 4, train_loss: 0.24, train_acc: 0.96
epoch: 5, train_loss: 0.20, train_acc: 0.97


## Testing

In [0]:
let stepsInEpoch = Int32(Float(testX.shape[0]) / Float(32))

var avgAcc:Float = 0.0 

for i in 0..<stepsInEpoch{
  let X = minibatch(in: testX, at: i)
  let y = minibatch(in: testY, at: i)
  
  let logits = model.applied(to: X, in: inferenceContext)
  let acc = getAccuracy(y:y, logits:logits)
  avgAcc += acc / Float(stepsInEpoch)
}

print("test_acc: \(avgAcc)")

License https://github.com/tensorflow/swift-models/blob/stable/LICENSE